In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key='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'
)
respon = llm.invoke("The first person to land on the moon was...")
print(respon.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2862181/software-development-engineer-fintech")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Development Engineer, FinTech - Job ID: 2862181 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Software Development Engineer, FinTechJob ID: 2862181 | ADCI HYD 13 SEZApply nowDESCRIPTIONAre you interested in building high-performance, globally scalable Financial systems that support Amazon's current and future growth? Are you seeking an environment where you can drive innovation leveraging the scalability and innovation with Amazon's AWS cloud services? Do you have a passion for ensuring a positive customer experience? This is the job for you.Amazon Financial Technology Team is looking for a results-oriented, driven software development engineer, who can help us create the next generation of distributed, scalable financial systems. Our ideal candidate thrives in a fast-paced e

In [40]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' : page_data})
print(res.content)

```json
{
  "role": "Software Development Engineer, FinTech",
  "experience": {
    "basic": {
      "years": 3,
      "qualifications": [
        "non-internship professional software development experience",
        "2+ years of non-internship design or architecture (design patterns, reliability and scaling) of new and existing systems experience",
        "3+ years of Video Games Industry (supporting title Development, Release, or Live Ops) experience",
        "Experience programming with at least one software programming language"
      ]
    },
    "preferred": {
      "years": 3,
      "qualifications": [
        "full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations experience",
        "Bachelor's degree in computer science or equivalent"
      ]
    }
  },
  "skills": {
    "programming_languages": ["at least one software programming language"],
    "experience": [
      "design or a

In [14]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Development Engineer, FinTech',
 'experience': {'basic': {'years': 3,
   'qualifications': ['non-internship professional software development experience',
    '2+ years of non-internship design or architecture (design patterns, reliability and scaling) of new and existing systems experience',
    '3+ years of Video Games Industry (supporting title Development, Release, or Live Ops) experience',
    'Experience programming with at least one software programming language']},
  'preferred': {'years': 3,
   'qualifications': ['full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations experience',
    "Bachelor's degree in computer science or equivalent"]}},
 'skills': {'programming_languages': ['at least one software programming language'],
  'experience': ['design or architecture (design patterns, reliability and scaling) of new and existing systems',
   'Video Games Industry (supp

In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [36]:
links = collection.query(query_texts=['Experience in python', "Experience in react"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [22]:
type('skills')

str

In [24]:
type(json_res)

dict

In [44]:
job

{'role': 'Software Development Engineer, FinTech',
 'experience': {'basic': {'years': 3,
   'qualifications': ['non-internship professional software development experience',
    '2+ years of non-internship design or architecture (design patterns, reliability and scaling) of new and existing systems experience',
    '3+ years of Video Games Industry (supporting title Development, Release, or Live Ops) experience',
    'Experience programming with at least one software programming language']},
  'preferred': {'years': 3,
   'qualifications': ['full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations experience',
    "Bachelor's degree in computer science or equivalent"]}},
 'skills': {'programming_languages': ['at least one software programming language'],
  'experience': ['design or architecture (design patterns, reliability and scaling) of new and existing systems',
   'Video Games Industry (supp

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your FinTech Potential with AtliQ

Dear Hiring Manager,

I came across the job description for a Software Development Engineer, FinTech at Amazon, and I was impressed by the exciting opportunity to build high-performance, globally scalable Financial systems. As a Business Development Executive at AtliQ, an AI & Software Consulting company, I'd like to introduce you to our capabilities in fulfilling your needs.

AtliQ has a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise spans a wide range of technologies, including machine learning, Python, React, and React Native.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities in FinTech and software development:

- **Machine Learning with Python**: Check out our portfolio on machine learning with Python at [https://example.com/ml-python-portfolio](http